### Query ROR for an organization and the hierarchy of sub-organizations below it

This notebook queries the [ROR API](https://ror.readme.io/) to construct a hierarchy of sub-organizations starting at a given organization. It takes a ROR URL or ID which is used to retrieve the organizations specified in the metadata field "`relationships`" with `"type"=Child` recursively to build a tree structure. The tree represents the hierarchy of sub-organizations and will be outputted.


In [ ]:
# Prerequisites:
!pip install requests
import requests                        # dependency to make HTTP calls
!pip install anytree
from anytree import Node, RenderTree   # dependency to construct tree structure

The input value for all following queries is a ROR URL or ID, e.g. '`https://ror.org/03vek6s52`' or '`03vek6s52`'.

In [2]:
# input parameter
example_ror="https://ror.org/03vek6s52"

We may use it to query the ROR API once for the organization's metadata...

In [3]:
# URL to ROR API
ROR_API_ENDPOINT = "https://api.ror.org/organizations"

# query ROR API for organization's metadata
def query_ror_api(ror):
    complete_url=requests.utils.requote_uri(ROR_API_ENDPOINT + "/" + ror)
    response = requests.get(url=complete_url,
                      headers={'Accept': 'application/json'})
    response.raise_for_status()
    result=response.json()
    return result


#---- example execution
# uncomment following lines to see the metadata for specified example_ror
#import pprint
#organization = query_ror_api(example_ror)
#pprint.pprint(organization)

But in this notebook we use it as a starting point to recursively query the ROR API using the relationship type "`Child`" to construct the organizational hierarchy below it.

In [4]:
# construct organizational tree recursively starting at given ROR
def construct_tree(ror, parent=None):
    organization = query_ror_api(ror)
    current_node = Node(organization["name"], parent=parent)

    for rel in organization['relationships']:
        if rel["type"]=="Child":
            construct_tree(rel["id"], current_node)

    return current_node


#---- example execution
organigram = construct_tree(example_ror)
print(RenderTree(organigram))

Node('/Harvard University')
├── Node('/Harvard University/Athinoula A. Martinos Center for Biomedical Imaging')
├── Node('/Harvard University/Berenson Allen Center for Noninvasive Brain Stimulation')
├── Node('/Harvard University/Center for Astrophysics Harvard & Smithsonian')
│   ├── Node('/Harvard University/Center for Astrophysics Harvard & Smithsonian/Harvard College Observatory')
│   └── Node('/Harvard University/Center for Astrophysics Harvard & Smithsonian/Smithsonian Astrophysical Observatory')
├── Node('/Harvard University/Center for Systems Biology')
├── Node('/Harvard University/Center for Vascular Biology Research')
├── Node('/Harvard University/Gordon Center for Medical Imaging')
├── Node('/Harvard University/Harvard Stem Cell Institute')
├── Node('/Harvard University/Harvard University Press')
├── Node('/Harvard University/MIT-Harvard Center for Ultracold Atoms')
├── Node('/Harvard University/Ragon Institute of MGH, MIT and Harvard')
├── Node('/Harvard University/Sleep an